In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns
import scipy.stats as stats
import os 
import math 

from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from sklearn.metrics import silhouette_score
from sklearn.decomposition import PCA
from sklearn.metrics import pairwise_distances_argmin_min
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

from sklearn import datasets
from sklearn.cluster import KMeans

import warnings
warnings.filterwarnings('ignore')

In [2]:
df=pd.read_csv('renewable_power_plants_DE.csv')
df.head()

,electrical_capacity,energy_source_level_1,energy_source_level_2,energy_source_level_3,technology,data_source,nuts_1_region,nuts_2_region,nuts_3_region,lon,...,postcode,address,federal_state,commissioning_date,decommissioning_date,voltage_level,eeg_id,dso,dso_id,tso
0,0.080,Renewable energy,Bioenergy,Sewage and landfill gas,Sewage and landfill gas,TransnetBW,DE1,DE13,DE135,8.578018,...,78727.0,NaN,Baden-Württemberg,1900-01-02,NaN,medium voltage,NaN,Netze BW GmbH,10000843.0,TransnetBW
1,0.015,Renewable energy,Hydro,NaN,Run-of-river,TenneT,DE2,DE25,DE25C,10.881562,...,91757.0,NaN,Brandenburg,1901-01-01,NaN,low voltage,NaN,Stadtwerke Treuchtlingen,10001282.0,TenneT
2,0.055,Renewable energy,Hydro,NaN,Run-of-river,TransnetBW,DE1,DE14,DE148,9.664273,...,88255.0,NaN,Baden-Württemberg,1901-01-01,NaN,low voltage / medium voltage,NaN,Netze BW GmbH,10000843.0,TransnetBW
3,0.120,Renewable energy,Hydro,NaN,Run-of-river,TransnetBW,DE1,DE14,DE148,9.664273,...,88255.0,NaN,Baden-Württemberg,1901-01-01,NaN,low voltage,NaN,Netze BW GmbH,10000843.0,TransnetBW
4,0.044,Renewable energy,Hydro,NaN,Run-of-river,TransnetBW,DE1,DE12,DE128,8.775571,...,69245.0,NaN,Baden-Württemberg,1901-07-01,NaN,low voltage,NaN,Syna GmbH,10001495.0,TransnetBW


In [3]:
df.columns

Index(['electrical_capacity', 'energy_source_level_1', 'energy_source_level_2',
       'energy_source_level_3', 'technology', 'data_source', 'nuts_1_region',
       'nuts_2_region', 'nuts_3_region', 'lon', 'lat', 'municipality',
       'municipality_code', 'postcode', 'address', 'federal_state',
       'commissioning_date', 'decommissioning_date', 'voltage_level', 'eeg_id',
       'dso', 'dso_id', 'tso'],
      dtype='object')

#### Creating SQL Schema

In [4]:
#df_sql = df.head(1000)

In [5]:
#df_sql.to_csv("C:/Users/Ouroboros/Desktop/renew_en_sql.csv", sep=";",header=True, index=False)

### Cleaning

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1768745 entries, 0 to 1768744
Data columns (total 23 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   electrical_capacity    float64
 1   energy_source_level_1  object 
 2   energy_source_level_2  object 
 3   energy_source_level_3  object 
 4   technology             object 
 5   data_source            object 
 6   nuts_1_region          object 
 7   nuts_2_region          object 
 8   nuts_3_region          object 
 9   lon                    float64
 10  lat                    float64
 11  municipality           object 
 12  municipality_code      float64
 13  postcode               float64
 14  address                object 
 15  federal_state          object 
 16  commissioning_date     object 
 17  decommissioning_date   float64
 18  voltage_level          object 
 19  eeg_id                 object 
 20  dso                    object 
 21  dso_id                 float64
 22  tso               

In [7]:
df.isna().sum()

electrical_capacity            0
energy_source_level_1          0
energy_source_level_2          0
energy_source_level_3    1754346
technology                     0
data_source                    0
nuts_1_region                675
nuts_2_region                675
nuts_3_region                675
lon                         1286
lat                         1286
municipality             1073178
municipality_code          33860
postcode                      16
address                  1764642
federal_state                  6
commissioning_date             1
decommissioning_date     1768745
voltage_level              33142
eeg_id                   1768227
dso                        33141
dso_id                     33900
tso                        33887
dtype: int64

In [8]:
df1 = df.drop(columns = ['energy_source_level_3', 'municipality', 'municipality_code', 'address', 'decommissioning_date', 'eeg_id', 'energy_source_level_1', 'dso_id'])
df1.head()

,electrical_capacity,energy_source_level_2,technology,data_source,nuts_1_region,nuts_2_region,nuts_3_region,lon,lat,postcode,federal_state,commissioning_date,voltage_level,dso,tso
0,0.080,Bioenergy,Sewage and landfill gas,TransnetBW,DE1,DE13,DE135,8.578018,48.292429,78727.0,Baden-Württemberg,1900-01-02,medium voltage,Netze BW GmbH,TransnetBW
1,0.015,Hydro,Run-of-river,TenneT,DE2,DE25,DE25C,10.881562,48.963994,91757.0,Brandenburg,1901-01-01,low voltage,Stadtwerke Treuchtlingen,TenneT
2,0.055,Hydro,Run-of-river,TransnetBW,DE1,DE14,DE148,9.664273,47.846131,88255.0,Baden-Württemberg,1901-01-01,low voltage / medium voltage,Netze BW GmbH,TransnetBW
3,0.120,Hydro,Run-of-river,TransnetBW,DE1,DE14,DE148,9.664273,47.846131,88255.0,Baden-Württemberg,1901-01-01,low voltage,Netze BW GmbH,TransnetBW
4,0.044,Hydro,Run-of-river,TransnetBW,DE1,DE12,DE128,8.775571,49.355539,69245.0,Baden-Württemberg,1901-07-01,low voltage,Syna GmbH,TransnetBW


In [9]:
df.energy_source_level_2.nunique() #dummyfying

5

In [10]:
num = df[['electrical_capacity']]
num.head()


transformer = StandardScaler().fit(num)
x_standardized = transformer.transform(num)

In [11]:
cat = df1.select_dtypes(object)
cat.head()

,energy_source_level_2,technology,data_source,nuts_1_region,nuts_2_region,nuts_3_region,federal_state,commissioning_date,voltage_level,dso,tso
0,Bioenergy,Sewage and landfill gas,TransnetBW,DE1,DE13,DE135,Baden-Württemberg,1900-01-02,medium voltage,Netze BW GmbH,TransnetBW
1,Hydro,Run-of-river,TenneT,DE2,DE25,DE25C,Brandenburg,1901-01-01,low voltage,Stadtwerke Treuchtlingen,TenneT
2,Hydro,Run-of-river,TransnetBW,DE1,DE14,DE148,Baden-Württemberg,1901-01-01,low voltage / medium voltage,Netze BW GmbH,TransnetBW
3,Hydro,Run-of-river,TransnetBW,DE1,DE14,DE148,Baden-Württemberg,1901-01-01,low voltage,Netze BW GmbH,TransnetBW
4,Hydro,Run-of-river,TransnetBW,DE1,DE12,DE128,Baden-Württemberg,1901-07-01,low voltage,Syna GmbH,TransnetBW


In [12]:
df1.isna().sum()

electrical_capacity          0
energy_source_level_2        0
technology                   0
data_source                  0
nuts_1_region              675
nuts_2_region              675
nuts_3_region              675
lon                       1286
lat                       1286
postcode                    16
federal_state                6
commissioning_date           1
voltage_level            33142
dso                      33141
tso                      33887
dtype: int64

In [13]:
cat = cat.apply(lambda x: x.replace("NaN", "0"))

In [14]:
#num = num.apply(lambda x: x.replace(np.NaN, 0))

In [15]:
cat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1768745 entries, 0 to 1768744
Data columns (total 11 columns):
 #   Column                 Dtype 
---  ------                 ----- 
 0   energy_source_level_2  object
 1   technology             object
 2   data_source            object
 3   nuts_1_region          object
 4   nuts_2_region          object
 5   nuts_3_region          object
 6   federal_state          object
 7   commissioning_date     object
 8   voltage_level          object
 9   dso                    object
 10  tso                    object
dtypes: object(11)
memory usage: 148.4+ MB


In [16]:
cat.isna().sum()

energy_source_level_2        0
technology                   0
data_source                  0
nuts_1_region              675
nuts_2_region              675
nuts_3_region              675
federal_state                6
commissioning_date           1
voltage_level            33142
dso                      33141
tso                      33887
dtype: int64

### Classification

In [17]:
cat_dum = cat[['technology', 'data_source', 'voltage_level', 'federal_state']]

In [18]:
cat_dum = pd.get_dummies(cat_dum)

In [19]:
df3 = pd.concat([cat_dum, num], axis=1)
df3.head()

,technology_Biomass and biogas,technology_Geothermal,technology_Offshore,technology_Onshore,technology_Photovoltaics,technology_Photovoltaics ground,technology_Run-of-river,technology_Sewage and landfill gas,data_source_50Hertz,data_source_Amprion,...,federal_state_Mecklenburg-Vorpommern,federal_state_Niedersachsen,federal_state_Nordrhein-Westfalen,federal_state_Rheinland-Pfalz,federal_state_Saarland,federal_state_Sachsen,federal_state_Sachsen-Anhalt,federal_state_Schleswig-Holstein,federal_state_Thüringen,electrical_capacity
0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0.080
1,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0.015
2,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0.055
3,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0.120
4,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0.044


In [20]:
#df3.isna().sum()

In [21]:
X = df3
y = df['energy_source_level_2']

In [22]:
#transformer = StandardScaler().fit(X)
#x_standardized = transformer.transform(X)

In [23]:
#d3_stand = pd.DataFrame(x_standardized,columns=df3.columns.tolist()[:-1])
#df3_stand.head()
# = df3_stand
# = df['energy_source_level_2']

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=100)

In [25]:
classification = LogisticRegression(random_state=100, max_iter=10000)
classification.fit(X_train, y_train)
predictions = classification.predict(X_test)
print(metrics.classification_report(y_test, predictions))

              precision    recall  f1-score   support

   Bioenergy       1.00      1.00      1.00      4410
  Geothermal       1.00      1.00      1.00         2
       Hydro       1.00      1.00      1.00      2090
       Solar       1.00      1.00      1.00    517450
        Wind       1.00      1.00      1.00      6672

    accuracy                           1.00    530624
   macro avg       1.00      1.00      1.00    530624
weighted avg       1.00      1.00      1.00    530624



In [26]:
classification.score(X_test, y_test)

1.0

In [27]:
y.value_counts()

Solar         1725093
Wind            22281
Bioenergy       14399
Hydro            6961
Geothermal         11
Name: energy_source_level_2, dtype: int64

### Managing Imbalance - Upsampling

In [28]:
solar_cat = cat[cat['energy_source_level_2'] == 'Solar']
wind_cat = cat[cat['energy_source_level_2'] == 'Wind']
bio_cat = cat[cat['energy_source_level_2'] == 'Bioenergy']
hydro_cat = cat[cat['energy_source_level_2'] == 'Hydro']
geo_cat = cat[cat['energy_source_level_2'] == 'Geothermal']

wind_cat1 = wind_cat.sample(len(solar_cat), replace=True)  
(wind_cat1.shape)

#len(bio_cat), len(hydro_cat), len(geo_cat),

(1725093, 11)

In [29]:
bio_cat1 = bio_cat.sample(len(solar_cat), replace=True)  
#(bio_cat1.shape)

In [30]:
hydro_cat1 = hydro_cat.sample(len(solar_cat), replace=True)  
#(hydro_cat1.shape)

In [31]:
geo_cat1 = geo_cat.sample(len(solar_cat), replace=True)  
#(geo_cat1.shape)

In [32]:
upsa = pd.concat([solar_cat, wind_cat1, bio_cat1, hydro_cat1, geo_cat1], axis=0)
#shuffling the data
upsa = upsa.sample(frac=1)
print(upsa['energy_source_level_2'].value_counts())

Solar         1725093
Bioenergy     1725093
Hydro         1725093
Geothermal    1725093
Wind          1725093
Name: energy_source_level_2, dtype: int64


In [33]:
upsa

,energy_source_level_2,technology,data_source,nuts_1_region,nuts_2_region,nuts_3_region,federal_state,commissioning_date,voltage_level,dso,tso
328410,Solar,Photovoltaics,50Hertz,DEG,DEG0,DEG0P,Thüringen,2007-11-23,low voltage,TEN Thüringer Energienetze GmbH & Co. KG,50Hertz
269094,Bioenergy,Biomass and biogas,TenneT,DE2,DE25,DE256,Brandenburg,2006-12-31,medium voltage,N-ERGIE Netz GmbH,TenneT
1494419,Hydro,Run-of-river,TenneT,DE2,DE27,DE27B,Brandenburg,2014-06-26,medium voltage,E-Werke Reutte GmbH & Co. KG,TenneT
1112736,Bioenergy,Biomass and biogas,TenneT,DE2,DE22,DE228,Brandenburg,2011-12-15,low voltage,Stadtwerke Vilshofen GmbH,TenneT
5280,Hydro,Run-of-river,50Hertz,DEG,DEG0,DEG0L,Thüringen,1995-12-18,medium voltage,TEN Thüringer Energienetze GmbH & Co. KG,50Hertz
...,...,...,...,...,...,...,...,...,...,...,...
563578,Geothermal,Geothermal,TransnetBW,DE1,DE12,DE123,Baden-Württemberg,2009-10-23,medium voltage,Energie- und Wasserversorgung Bruchsal GmbH,TransnetBW
1359268,Geothermal,Geothermal,TenneT,DE2,DE21,DE21H,Brandenburg,2013-03-25,medium voltage,Bayernwerk Netz GmbH,TenneT
83690,Bioenergy,Biomass and biogas,Amprion,DE7,DE71,DE714,Hessen,2003-10-22,medium voltage,InfraServ GmbH & Co. Wiesbaden KG,Amprion
146,Hydro,Run-of-river,TenneT,DE7,DE71,DE71E,Hamburg,1924-01-01,medium voltage,ovag Netz GmbH,TenneT


In [49]:
#upsa.isna().sum()
upsa1.isna().sum()

energy_source_level_2    0
technology               0
data_source              0
federal_state            0
commissioning_date       0
dtype: int64

In [36]:
upsa1 = upsa.drop(columns = ['nuts_1_region', 'nuts_2_region', 'nuts_3_region', 'voltage_level', 'dso', 'tso'])
upsa1.head()

,energy_source_level_2,technology,data_source,federal_state,commissioning_date
328410,Solar,Photovoltaics,50Hertz,Thüringen,2007-11-23
269094,Bioenergy,Biomass and biogas,TenneT,Brandenburg,2006-12-31
1494419,Hydro,Run-of-river,TenneT,Brandenburg,2014-06-26
1112736,Bioenergy,Biomass and biogas,TenneT,Brandenburg,2011-12-15
5280,Hydro,Run-of-river,50Hertz,Thüringen,1995-12-18


In [47]:
upsa1[['federal_state', 'commissioning_date']] = upsa1[['federal_state', 'commissioning_date']].fillna(0)
#upsa1.replace('nan', np.NaN, inplace=True) # nan could be string
#upsa1 = upsa1.apply(lambda x: x.replace("NaN", "0"))

In [48]:
upsa1.federal_state.unique()

array(['Thüringen', 'Brandenburg', 'Rheinland-Pfalz',
       'Nordrhein-Westfalen', 'Niedersachsen', 'Sachsen-Anhalt',
       'Saarland', 'Mecklenburg-Vorpommern', 'Baden-Württemberg',
       'Bayern', 'Hamburg', 'Hessen', 'Berlin ', 'Sachsen',
       'Ausschließliche Wirtschaftszone', 'Bremen', 'Baden-Würtemberg',
       'Berlin', 'Schleswig-Holstein', 0, 'Ausland'], dtype=object)

In [50]:
y1 = upsa1['energy_source_level_2']
# The Features
X1 = df3

In [53]:
transformer = StandardScaler().fit(X1)
X_stan = transformer.transform(X1)
X_stan.shape

(1768745, 42)

In [54]:
X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1, test_size=0.3, random_state=100)

ValueError: Found input variables with inconsistent numbers of samples: [1768745, 8625465]

In [55]:
#different lengths